In [2]:
import pandas 

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 3, 7, Finished, Available)

In [3]:
spark

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 3, 8, Finished, Available)

# Set up data configuration

In [4]:
blob_account_name = "marckvnonprodblob"
blob_container_name = "bigdata"
# read only
blob_sas_token = "?sv=2021-10-04&st=2023-10-04T01%3A42%3A59Z&se=2024-01-02T02%3A42%3A00Z&sr=c&sp=rlf&sig=w3CH9MbCOpwO7DtHlrahc7AlRPxSZZb8MOgS6TaXLzI%3D"

wasbs_base_url = (
    f"wasbs://{blob_container_name}@{blob_account_name}.blob.core.windows.net/"
)
spark.conf.set(
    f"fs.azure.sas.{blob_container_name}.{blob_account_name}.blob.core.windows.net",
    blob_sas_token,
)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 3, 9, Finished, Available)

# Read the whole Reddit data

In [5]:
comments_path = "reddit-parquet/comments/"
submissions_path = "reddit-parquet/submissions/"

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 3, 10, Finished, Available)

In [6]:
comments_df = spark.read.parquet(f"{wasbs_base_url}{comments_path}")
submissions_df = spark.read.parquet(f"{wasbs_base_url}{submissions_path}")

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 3, 11, Finished, Available)

## Get sales data from local csv file

In [39]:
import pandas as pd 
A = "Users/yc1063/fall-2023-reddit-project-team-11/data/csv/vgsales.csv"
df = pd.read_csv(A,encoding ="utf-8")
dfs = df[["Name","NA_Sales","Global_Sales"]]
df

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 3, 44, Finished, Available)

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


# filter the subreddit that only contains popular game titles

In [8]:
from pyspark.sql.functions import col, asc,desc

submissions_filtered = submissions_df.filter(col("subreddit").isin(df['Name'].tolist()))

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 3, 13, Finished, Available)

In [9]:
submissions_pop_games = submissions_filtered.groupBy("subreddit").count().orderBy(col("count"), ascending=False).collect()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 3, 14, Finished, Available)

In [11]:
submissions_pop_games_df = spark.createDataFrame(submissions_pop_games).toPandas()
submissions_pop_games_df

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 3, 16, Finished, Available)

,subreddit,count
0,Minecraft,903622
1,Overwatch,297011
2,Sneakers,256072
3,Terraria,221004
4,Destiny,130433
...,...,...
150,GunPey,1
151,Malice,1
152,Shox,1
153,Kamiwaza,1


In [29]:
dft = dfs.merge(submissions_pop_games_df, how = "inner",left_on=["Name"],right_on=["subreddit"])
dft

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 3, 34, Finished, Available)

,Name,NA_Sales,Global_Sales,subreddit,count
0,Tetris,23.20,30.26,Tetris,9049
1,Tetris,2.97,5.58,Tetris,9049
2,Nintendogs,9.07,24.76,Nintendogs,87
3,Minecraft,5.58,9.20,Minecraft,903622
4,Minecraft,1.97,5.42,Minecraft,903622
...,...,...,...,...,...
278,Contrast,0.00,0.02,Contrast,3
279,Uppers,0.00,0.01,Uppers,11
280,Rewrite,0.00,0.01,Rewrite,767
281,Trine,0.00,0.01,Trine,130


In [34]:
dft["Name"]= dft["Name"].astype(str)
agg_df = dft.groupby('Name').agg({'NA_Sales': 'sum', 'Global_Sales': 'sum'}).reset_index()

# Merge the aggregated DataFrame with the original 'Count' column
result_df = dft.drop(columns=['NA_Sales', 'Global_Sales']).merge(agg_df, on='Name').drop_duplicates(subset='Name')
result_df

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 3, 39, Finished, Available)

,Name,subreddit,count,NA_Sales,Global_Sales
0,Tetris,Tetris,9049,26.17,35.84
2,Nintendogs,Nintendogs,87,9.07,24.76
3,Minecraft,Minecraft,903622,10.82,23.73
9,Destiny,Destiny,130433,6.59,12.61
13,Asteroids,Asteroids,1,4.64,5.46
...,...,...,...,...,...
278,Contrast,Contrast,3,0.00,0.02
279,Uppers,Uppers,11,0.00,0.01
280,Rewrite,Rewrite,767,0.00,0.01
281,Trine,Trine,130,0.00,0.01


In [35]:
import os
CSV_DIR = os.path.join("Users/yc1063/fall-2023-reddit-project-team-11/data", "csv")

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 3, 40, Finished, Available)

In [36]:
result_df.to_csv(f"{CSV_DIR}/game_sales.csv", index=False)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 3, 41, Finished, Available)

# show the popularity of each popular game each month

In [37]:
submissions_filtered_ym = submissions_filtered.select("subreddit", "year").cache()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 3, 42, Finished, Available)

In [38]:
submissions_filtered_ym_pop = submissions_filtered_ym.groupBy("subreddit","year").count().orderBy(col("count"), ascending=False).collect()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 3, 43, Finished, Available)

In [40]:
submissions_filtered_ym_pop_df = spark.createDataFrame(submissions_filtered_ym_pop).toPandas()
submissions_filtered_ym_pop_df

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 3, 45, Finished, Available)

,subreddit,year,count
0,Minecraft,2021,549210
1,Minecraft,2022,294871
2,Overwatch,2022,168468
3,Sneakers,2021,117746
4,Sneakers,2022,111947
...,...,...,...
386,Darkstone,2023,1
387,Viz,2022,1
388,TwinBee,2023,1
389,Kamiwaza,2022,1


In [23]:
submissions_filtered_ym_pop_df.to_csv(f"{CSV_DIR}/top_10_pop_games_month.csv", index=False)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 0, 28, Finished, Available)